### Climate Policymaking : a decision-making problem about CO2 mitigation

dapted from EMA Workbench Lakeproblem Example

The Climate Change Problem forms a hypothetical scenario where the global community makes the decision on the amount of annual CO2 emissions to be released. The challenge is, if the CO2 concentration in the atmosphere surpasses a certain critical threshold or a "tipping point", it may cause irreversible and dangerous changes to the global climate system.

The Climate Change model has 4 **outcome indicators**:

1. **CO2_concentration**: The average concentration of CO2 in the atmosphere over a given timeframe, which we aim to minimize.

2. **economic_welfare**: The economic benefits derived from emitting CO2 minus the costs of having excessive CO2 in the atmosphere, which we aim to maximize.

3. **emission_inertia**: Inertia is maximized by setting an annual reduction limit on emissions, thereby preventing rapid declines in emissions that would require massive investments, and measured by the fraction of years where emission reductions stay below the set limit.

4. **reliability**: The fraction of years when the CO2 concentration in the atmosphere is below the critical threshold, which we aim to maximize to prevent triggering dangerous climate disruptions.

The Climate Change Problem is characterized by both stochastic uncertainty and **deep uncertainty**. The stochastic uncertainty arises from the natural emissions of CO2. In order to reduce this form of uncertainty, multiple replications are performed, and the average over these replications is taken. Deep uncertainty involves uncertainty about the mean and standard deviation of the log-normal distribution representing natural emissions, Earth's natural CO2 absorption rate, the natural emission rate of CO2, and the discount rate. Based on scientific understanding, ranges for these deeply uncertain factors can be established, as well as their best estimate or default values.

|Parameter	|Range	        |Default value| Description |
|-----------|--------------:|------------:|---------------------------:|
|$\mu$    	|0.01 – 0.05	|0.02         | Mean of the log-normal distribution representing the natural variability in CO2 emissions |
|$\sigma$	|0.001 – 0.005 	|0.0017       | Standard deviation of the log-normal distribution representing the natural variability in CO2 emissions |
|$b$      	|0.1 – 0.45	    |0.42         | Earth’s natural CO2 absorption rate, representing the rate at which CO2 is removed from the atmosphere |
|$q$	    |2 – 4.5	    |2            | Represents a decay rate in the concentration of CO2 in the atmosphere due to oceanic absorption |
|$\delta$	|0.93 – 0.99	|0.98         | Discount rate for the economic welfare calculation, indicating the decrease in future benefits compared to present benefits |

we apply a more sophisticated strategy to the problem of climate change. We employ a **closed-loop** version of the model, which means that anthropogenic emissions, represented as $a_t$, is dependent on $X_t$ (the CO2 concentration in the atmosphere at time t). For instance, we can lowerCO2 emissions when approaching the critical threshold. Using "cubic radial basis functions", following Quinn et al. 2017 we formulate $a_t$ as follows: (Please note that the formulation of $a_t$ will depend on the policy and decision mechanisms introduced, which could include a range of tools from emission tax, cap-and-trade schemes to regulations).


\begin{equation}
    a_{t} =  min\Bigg(max\bigg(\sum\limits_{j=1}^{n} w_{j}\left\vert{\frac{X_{t,i}-c_{j}}{r_{j}}}\right\vert^3, 0.01\bigg), 0.1\Bigg) \\
    s.t. \\
    -2 \leq c_{j} \leq 2 \\
    0 \leq r_{j} \leq 2 \\ 
    0 \leq w_{j} \leq 1 \\
    \sum\limits_{j=1}^{n} w_{j} = 1
\end{equation}


The parameters that define this function also define the emissions strategy over time. Hence, the decision **levers** are the five parameters of this functions, namely $c_1$, $c_2$, $r_1$, $r_2$ and $w_1$. ($w_2$ = 1 - $w_1$).

Note:: i is index for the realization, given m realizations; j is the index for the radial basis function, given 2 radial basis functions. 

In [ ]:
#Importing the required packages

from ema_workbench import RealParameter, ScalarOutcome, Constant, Model
#from optimization_lake_model_dps import lake_problem
from model.simple_climate_economy_model import climate_policy_model
from ema_workbench import MultiprocessingEvaluator, ema_logging, perform_experiments
ema_logging.log_to_stderr(ema_logging.INFO)

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from ema_workbench import MultiprocessingEvaluator, ema_logging
from ema_workbench.em_framework.evaluators import BaseEvaluator

from ema_workbench.analysis import pairs_plotting
from ema_workbench.analysis import parcoords



## Setting up the model in XLRM framework for  Exploratory Analysis

In [ ]:
# Use the XLRM framework to set up the exploration of the model

model = Model("climatepolicymodel", function=climate_policy_model)

# specify uncertainties
model.uncertainties = [
    RealParameter("b", 0.1, 0.45),
    RealParameter("q", 2.0, 4.5),
    RealParameter("mean", 0.01, 0.05),
    RealParameter("stdev", 0.001, 0.005),
    RealParameter("delta", 0.93, 0.99),
]

# set levers
model.levers = [
    RealParameter("c1", -2, 2),
    RealParameter("c2", -2, 2),
    RealParameter("r1", 0, 2),
    RealParameter("r2", 0, 2),
    RealParameter("w1", 0, 1),
]

# specify outcomes
model.outcomes = [
    ScalarOutcome("max_CO2"),
    ScalarOutcome("economic_welfare"),
    ScalarOutcome("emission_inertia"),
    ScalarOutcome("reliability"),
]

#TODO remove this later
# override some of the defaults of the model
model.constants = [
    Constant("alpha", 0.41),
    Constant("nsamples", 150),
    Constant("myears", 100),
]


## Open Exploration

In [ ]:

with MultiprocessingEvaluator(model) as evaluator:
    # Run 1000 scenarios for 10 policies
    #Can play around with number of Scenarios and Policies
    #Note: Large numbers of scenarios and policies will take a long time to run
    experiments, outcomes = evaluator.perform_experiments(scenarios=1000, policies=10)

### Plotting the different possible outcomes arising from combinations of the decision levers and uncertainties

In [ ]:

fig, axes = pairs_plotting.pairs_scatter(experiments, outcomes, group_by="policy", legend=False)
fig.set_size_inches(15, 15)
plt.show()

### Searching for Pareto optimal solutions using MOEA

In [ ]:
search_resolution = 0.1

# specify outcomes
model.outcomes = [
    ScalarOutcome("max_CO2", ScalarOutcome.MINIMIZE),
    ScalarOutcome("economic_welfare", ScalarOutcome.MAXIMIZE),
    ScalarOutcome("emission_inertia", ScalarOutcome.MAXIMIZE),
    ScalarOutcome("reliability", ScalarOutcome.MAXIMIZE),
]

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=5e3, searchover='levers',
                                 epsilons=[search_resolution,]*len(model.outcomes))

### Plotting the trade-offs between the different objectives in Parallel Plot

In [ ]:
data = results.loc[:, [o.name for o in model.outcomes]]
limits = parcoords.get_limits(data)
limits.loc[0, ['economic_welfare', 'emission_inertia', 'reliability', 'max_CO2']] = 0


paraxes = parcoords.ParallelAxes(limits, rot = 45)
paraxes.plot(data)
paraxes.invert_axis('max_CO2')


# Change the size of the plot
paraxes.fig.set_size_inches(20, 14)

plt.show()



## Step 3: Select candidate solutions under uncertainty

We now have a large number of candidate solutions (policies), we can re-evaluate them over the various deeply uncertain factors to assess their robustness against uncertainties.

For this robustness evaluation, we need to explore the scenarios for each solution. It means that, if we would like to run for instance 1000 scenarios for each solution, we might have to execute a very large number of runs.

Here, to simplify the case, let's suppose that decision makers have a hard constrain on *reliability*. No solution with less than 90% reliability is acceptable for them. Therefore, we can reduce the size of the solution set according to this constraint. 

**Apply this constraint of reliability on the results, and create a new dataframe named new_reults**


There are various ways to do it. One way is to use logical indexing. Basically, create a boolean vector that indicates for each row if the constraint is met or not. Next, we can use this as an index on the dataframe to get only the rows for which the index is true.




In [ ]:
# Selecting the policies based on specific criteria

logical = (results.reliability > 0.6 ) & (results.max_CO2 < 0.2) & (results.economic_welfare > 0.4)
np.sum(logical)

In [ ]:
data = results[logical]
data = data.loc[:, [o.name for o in model.outcomes]]
limits = parcoords.get_limits(data)
limits.loc[0, ['economic_welfare', 'emission_inertia', 'reliability', 'max_CO2']] = 0


paraxes = parcoords.ParallelAxes(limits, rot = 45)
paraxes.plot(data)
paraxes.invert_axis('max_CO2')


# Change the size of the plot
paraxes.fig.set_size_inches(20, 14)

In [ ]:
results[logical]

In [ ]:
policies = results[logical]
policies = policies.drop([o.name for o in model.outcomes], axis=1)
policies